In [ ]:
import os
import pandas as pd
import requests
from io import StringIO
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Import all Children with Birthdate and School District from report
# Report Name: MC - Early Kinder Eligibility by School District
# Report Url: https://admin.upk.colorado.gov/reports/709ed9e0-6255-49c5-8504-9acbca5e6a2b
bridgecare_shared_report_url = os.getenv("API_URL")
report_id = os.getenv("REPORT_ID")
bc_params = {'share_token': os.getenv("SHARE_TOKEN")}

response = requests.get(bridgecare_shared_report_url +
                        report_id, params=bc_params)

students_df = pd.read_csv(StringIO(response.text), sep=",")
students_df = students_df.rename(
    columns={
        "Group.School District": "School District",
        "Group.ID": "Group ID"

    }
)

# count of rows
print(students_df.shape[0])

students_df.head()


In [ ]:
# Import School District Cutoff data
# make sure school district names map to values in "Group.School District"
school_district_df = pd.read_csv(
    "../../data_in/school_districts/School District Cutoffs.csv")
school_district_df = school_district_df.rename(
    columns={
        "NAME": "School District"
    }
)
school_district_df.head()

In [ ]:
students_merged_df = students_df.merge(
    school_district_df, on="School District")
students_merged_df = students_merged_df.sort_values(by=['Date of birth'])
students_merged_df.head(10)

In [ ]:
# Calculate Pre-Kinde eligibility based on 
# school district cutoff date and birthdate.
def calc_eligibility(row):
    # If their school district doesn't have a cutoff date, they are not eligible
    if pd.isna(row["Cutoff date"]):
        return None
    else:
        # Define key dates
        kinder_eligible_date = datetime.strptime(row["Cutoff date"] + "-2018", '%d-%b-%Y').date()
        upk_4yo_eligible_date = datetime.strptime(row["Cutoff date"] + "-2019", '%d-%b-%Y').date()
        upk_3yo_eligible_date = datetime.strptime(row["Cutoff date"] + "-2020", '%d-%b-%Y').date()
        threeyear_before_kinder_date = datetime.strptime(row["Cutoff date"] + "-2021", '%d-%b-%Y').date()
        school_age_date = pd.to_datetime(
            kinder_eligible_date)+pd.DateOffset(years=-1)
        
        dob_string = str(row["Date of birth"])

        # If the date of birth is missing, return None
        if dob_string is None:
            return None
        if dob_string == 'nan':
            return None    

        # Format date of birth to date type 
        dob = datetime.strptime(dob_string, '%Y-%m-%d').date()

        # Start with the oldest age group and work down
        if dob <= school_age_date:
            return None
        elif dob <= kinder_eligible_date:
            return "Kinder"
        elif dob <= upk_4yo_eligible_date:
            return "1 Year Before Kinder"
        elif dob <= upk_3yo_eligible_date:
            return "2 Years Before Kinder"
        elif dob <= threeyear_before_kinder_date:
            return "3 Years Before Kinder"
        else:
            return None

students_merged_df['Early Kinder Eligibility'] = students_merged_df.apply(
    lambda row: calc_eligibility(row), axis=1)

students_merged_df = students_merged_df.sort_values(
    by=['ID'])
students_merged_df.head(10)

In [ ]:
students_merged_df.to_csv(
    '../../data_out/bridgecare/child/early_kinder_eligibility.csv', index=False)

In [ ]:
# Only keep ID and Early Kinder Eligibility columns for data sync
minified_df = students_merged_df[['ID', 'Early Kinder Eligibility', 'Group ID']]
# Skip rows with no pre-kinder eligibility status
minified_df = minified_df.dropna(thresh=2)
minified_df.to_csv(
    '../../data_out/bridgecare/child/early_kinder_eligibility_min.csv', index=False)